In [1]:
from modeling.gen6.api import symbolic, sqrt
from modeling.gen7.api7 import MFunctionalSet, MFunctionalSetLeaf
from engine.torchdata import print_formatted_table, load_vals, ExpandVector
import numpy as np
from scipy import optimize
import torch

In [2]:
A, Pb, Pc, D, h, R = symbolic('A', 'Pb', 'Pc', 'D', 'h', 'R')
D1 = MFunctionalSetLeaf(A == (Pb+Pc)/483.307, idvals=[0])
D2 = MFunctionalSetLeaf(Pb == 10*sqrt(Pc), idvals=[1])
D3 = MFunctionalSetLeaf(Pc == R*h**2/D**2*1e-18, idvals=[2])
FPF = MFunctionalSet(D1,D2,D3).subsetof(R>=8e6, h>=400e3).minimize(
    2500*D**2+12000*(A+1)+100*Pb)
FPF_MDF = FPF.config(elim=[D3, D2, D1])

In [3]:
# rearrange and specify new order
output_set = {0: Pb, 1: Pc}
elim_order = [{0}, {1, 2}]
FPF.config_from_order(elim_order)
#FPF.reconfigure(output_set)
# FPFr = FPF.reconfigure(output_set)
# FPFr.supersets[2].supersets

In [ ]:
x0_full = {'A': 1, 'Pb': 10, 'Pc': 10, 'D': 1, 'h': 100e3, 'R': 1}
obj, dobj, xguess, cons, idxs, solidxs = FPF_MDF.build_opt(x0=x0_full)

In [ ]:
xsol = optimize.minimize(obj, xguess, jac=dobj, 
                         constraints=cons, method='SLSQP')

In [ ]:
idxrev = {var.item():key for key,var in idxs.items()}
indices_in = {idxrev[val.item()]:torch.tensor([idx]) 
              for idx, val in enumerate(solidxs)}
ev = ExpandVector(indices_in, idxs)
xsolfull = FPF_MDF.build().analysis(ev(torch.tensor(xsol.x)))

In [ ]:
print_formatted_table([np.round(xsolfull,2)], idxs, idxrev)